# 🚗 YOLOv11 Car Damage Detection Training

This notebook trains a YOLOv11 segmentation model for car damage detection using your converted dataset.

**Classes**: car-damage, crack, dent, glass shatter, lamp broken, scratch, tire flat

**Requirements**:
1. Upload your `car_damage_yolo_dataset` folder to Google Drive
2. Enable GPU runtime (Runtime → Change runtime type → GPU)

In [ ]:
# 🔧 Setup and Installation
!pip install ultralytics

# Check GPU
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'No GPU'}")

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Navigate to your dataset
import os
os.chdir('/content/drive/My Drive/YOLOv11_Car_Damage')
print("Working directory:", os.getcwd())

In [ ]:
# 🔍 Verify Dataset
from pathlib import Path

dataset_path = Path('car_damage_yolo_dataset')

print("📁 Dataset Verification:")
dirs = ['images/train', 'images/val', 'labels/train', 'labels/val']

for dir_name in dirs:
    dir_path = dataset_path / dir_name
    if dir_path.exists():
        count = len(list(dir_path.glob('*')))
        print(f"✅ {dir_name}: {count} files")
    else:
        print(f"❌ {dir_name}: Not found")

# Check data.yaml
yaml_path = dataset_path / 'data.yaml'
if yaml_path.exists():
    print("✅ data.yaml found")
    with open(yaml_path, 'r') as f:
        print("\n📄 Configuration:")
        print(f.read())
else:
    print("❌ data.yaml not found")

In [ ]:
# 🔧 Update data.yaml for Colab
import yaml

# Update paths for Colab environment
config_content = f"""# YOLOv11 dataset configuration
path: /content/drive/My Drive/YOLOv11_Car_Damage/car_damage_yolo_dataset
train: images/train
val: images/val
test: images/test

# Classes
nc: 7
names: ['car-damage', 'crack', 'dent', 'glass shatter', 'lamp broken', 'scratch', 'tire flat']
"""

with open('car_damage_yolo_dataset/data.yaml', 'w') as f:
    f.write(config_content)

print("✅ Updated data.yaml for Colab")

In [ ]:
# 🧪 Test Training (1 epoch)
from ultralytics import YOLO

os.makedirs('models', exist_ok=True)

print("🏁 Test Training (1 epoch)...")

# Initialize model
model = YOLO('yolo11n-seg.pt')

# Test training
results = model.train(
    data='car_damage_yolo_dataset/data.yaml',
    epochs=1,
    imgsz=640,
    batch=8,
    name='car_damage_test',
    project='models',
    device=0
)

print("✅ Test completed! Ready for full training.")

In [ ]:
# 🚀 Full Training
print("🚀 Starting Full Training...")

# Initialize fresh model
model = YOLO('yolo11n-seg.pt')

# Full training
results = model.train(
    data='car_damage_yolo_dataset/data.yaml',
    epochs=100,        # Adjust as needed
    imgsz=640,
    batch=16,          # Reduce if GPU memory issues
    name='car_damage_full',
    project='models',
    patience=20,
    device=0,
    workers=2,
    cache=True,
    verbose=True
)

print("🎉 Training completed!")

In [ ]:
# 📊 View Training Results
from IPython.display import Image, display
import glob

results_dir = 'models/car_damage_full'

if os.path.exists(results_dir):
    print("📈 Training Results:")
    
    # Display training curves
    results_img = f"{results_dir}/results.png"
    if os.path.exists(results_img):
        display(Image(results_img))
    
    # Show validation images
    val_images = glob.glob(f"{results_dir}/val_*.jpg")
    for img_path in val_images[:3]:
        print(f"📸 {os.path.basename(img_path)}")
        display(Image(img_path))
else:
    print("⚠️ Results not found")

In [ ]:
# ✅ Validate Model
best_model_path = 'models/car_damage_full/weights/best.pt'

if os.path.exists(best_model_path):
    model = YOLO(best_model_path)
    
    print("🔬 Validating model...")
    val_results = model.val(
        data='car_damage_yolo_dataset/data.yaml'
    )
    
    print("📊 Validation Metrics:")
    print(f"mAP50: {val_results.box.map50:.4f}")
    print(f"mAP50-95: {val_results.box.map:.4f}")
else:
    print("❌ Best model not found")

In [ ]:
# 🧪 Test Inference
if os.path.exists(best_model_path):
    # Get test images
    test_images = list(Path('car_damage_yolo_dataset/images/val').glob('*'))[:3]
    
    model = YOLO(best_model_path)
    
    print("🖼️ Testing inference:")
    
    for img_path in test_images:
        print(f"Processing: {img_path.name}")
        
        # Run inference
        results = model(str(img_path))
        
        # Display results
        for result in results:
            result.show()
            
    print("✅ Inference test completed!")
else:
    print("⚠️ Model not available for inference")

In [ ]:
# 💾 Download Results
import shutil
from google.colab import files

print("📦 Preparing download...")

# Create results package
package_name = 'car_damage_results'
os.makedirs(package_name, exist_ok=True)

# Copy important files
if os.path.exists('models/car_damage_full/weights/best.pt'):
    shutil.copy('models/car_damage_full/weights/best.pt', 
               f'{package_name}/best_model.pt')
    print("✅ Copied best model")

if os.path.exists('models/car_damage_full/results.png'):
    shutil.copy('models/car_damage_full/results.png', 
               f'{package_name}/training_curves.png')
    print("✅ Copied training curves")

# Create summary
with open(f'{package_name}/summary.txt', 'w') as f:
    f.write("Car Damage Detection - YOLOv11 Results\n")
    f.write("=" * 40 + "\n")
    f.write("Model: YOLOv11n-seg\n")
    f.write("Classes: 7 (car-damage, crack, dent, glass shatter, lamp broken, scratch, tire flat)\n")
    f.write("Training: 9600 images\n")
    f.write("Validation: 800 images\n")

# Zip and download
shutil.make_archive(package_name, 'zip', package_name)
files.download(f'{package_name}.zip')

print("🎉 Download ready! Check your downloads folder.")

## 🎯 Usage Instructions

### Before Running:
1. Upload your `car_damage_yolo_dataset` folder to Google Drive in `/My Drive/YOLOv11_Car_Damage/`
2. Enable GPU: Runtime → Change runtime type → Hardware accelerator: GPU

### Training Tips:
- **Start with test training** (1 epoch) to verify everything works
- **Monitor GPU usage** to avoid timeouts
- **Adjust batch size** if you get memory errors
- **Save frequently** - Colab can disconnect

### Expected Results:
- **Training time**: 2-4 hours on free GPU
- **Model accuracy**: mAP50 around 0.6-0.8
- **Model size**: ~6MB

### After Training:
- Download the results zip file
- Use `best_model.pt` for inference
- Check `training_curves.png` for training progress

**Happy Training! 🚀**